# Basic setup

In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker
import requests
import json
from elasticsearch import Elasticsearch


In [2]:
res = requests.get('http://localhost:9200')
print(res.content)
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

b'{\n  "name" : "-L9u61j",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "7yPgKxlWRwmSpPqu9E9w2A",\n  "version" : {\n    "number" : "5.6.3",\n    "build_hash" : "1a2f265",\n    "build_date" : "2017-10-06T20:33:39.012Z",\n    "build_snapshot" : false,\n    "lucene_version" : "6.6.1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [45]:
print(requests.get('http://localhost:9200/_cat/health?v').text)
print(requests.get('http://localhost:9200/_cat/nodes?v').text)
print(requests.get('http://localhost:9200/_cat/shards?v').text)
print(requests.get('http://localhost:9200/_cat/indices?v').text)

epoch      timestamp cluster       status node.total node.data shards pri relo init unassign pending_tasks max_task_wait_time active_shards_percent
1509882574 14:49:34  elasticsearch yellow          1         1      6   6    0    0        6             0                  -                 50.0%

ip        heap.percent ram.percent cpu load_1m load_5m load_15m node.role master name
127.0.0.1            9          67  11    0.74    0.76     0.72 mdi       *      -L9u61j

index     shard prirep state      docs  store ip        node
zadolbali 2     p      STARTED       0   162b 127.0.0.1 -L9u61j
zadolbali 2     r      UNASSIGNED                       
zadolbali 4     p      STARTED       0   162b 127.0.0.1 -L9u61j
zadolbali 4     r      UNASSIGNED                       
zadolbali 1     p      STARTED       0   162b 127.0.0.1 -L9u61j
zadolbali 1     r      UNASSIGNED                       
zadolbali 3     p      STARTED       0   162b 127.0.0.1 -L9u61j
zadolbali 3     r      UNASSIGNED      

# Delete zadolbali index

In [41]:
r = requests.delete('http://localhost:9200/zadolbali') 
print(r.text)

{"acknowledged":true}


In [42]:
r = requests.get('http://localhost:9200/_cat/indices')
print(r.text)

yellow open .kibana L5hEXnBERGe6czr2OJchfg 1 1 2 1 10.8kb 10.8kb



# Create index

In [43]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

r = requests.put('http://localhost:9200/zadolbali', headers=headers, data=json.dumps({
    "mappings": {
        "story": {
            "properties": {
                "id": { "type": "integer" },
                "title":  { "type": "text" },
                "text": { "type": "text" },
                "published": {
                    "type": "date",
                    "format": "yyyyMMdd"
                    },
                "likes": { "type": "integer" },
                "tags": { "type": "text" },
                "url": { "type": "text" }
            }
        }
    }
}))
print(r.text)

{"acknowledged":true,"shards_acknowledged":true,"index":"zadolbali"}


In [44]:
print(requests.get('http://localhost:9200/zadolbali/_mapping').text)

{"zadolbali":{"mappings":{"story":{"properties":{"id":{"type":"integer"},"likes":{"type":"integer"},"published":{"type":"date","format":"yyyyMMdd"},"tags":{"type":"text"},"text":{"type":"text"},"title":{"type":"text"},"url":{"type":"text"}}}}}}


# Get data

In [47]:
class Story(object):
    pass
 
def loadSession():
    dbPath = '../corpus/stories.sqlite'
    engine = create_engine('sqlite:///%s' % dbPath, echo=True)
 
    bookmarks = Table('stories', MetaData(engine), autoload=True)
    mapper(Story, bookmarks)
 
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

session = loadSession()

2017-11-05 14:51:02,991 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-11-05 14:51:02,992 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 14:51:02,993 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-11-05 14:51:02,994 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 14:51:02,996 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stories")
2017-11-05 14:51:02,997 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 14:51:03,000 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'stories' AND type = 'table'
2017-11-05 14:51:03,001 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 14:51:03,004 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("stories")
2017-11-05 14:51:03,005 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 14:51:03,007 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (

In [48]:
stories = session.query(Story).all()
print(len(stories))
print(dir(stories[0]))

2017-11-05 14:51:04,608 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-05 14:51:04,612 INFO sqlalchemy.engine.base.Engine SELECT stories.id AS stories_id, stories.title AS stories_title, stories.published AS stories_published, stories.tags AS stories_tags, stories.text AS stories_text, stories.likes AS stories_likes, stories.hrefs AS stories_hrefs, stories.url AS stories_url 
FROM stories
2017-11-05 14:51:04,616 INFO sqlalchemy.engine.base.Engine ()
23558
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_sa_class_manager', '_sa_instance_state', 'hrefs', 'id', 'likes', 'published', 'tags', 'text', 'title', 'url']


# Index it!

In [49]:
# 'hrefs', 'id', 'likes', 'published', 'tags', 'text', 'title', 'url'
for story in stories[:100]:
    body = {
        "id": story.id,
        "title": story.title,
        "text": story.text,
        "published": story.published,
        "likes": story.likes,
        "tags": story.tags,
        "url": story.url
    }
    es.index(index='zadolbali', doc_type='stories', id=story.id, body=body)

In [50]:
es.get(index='zadolbali', id=5)

{'_id': '5',
 '_index': 'zadolbali',
 '_source': {'id': 5,
  'likes': 0,
  'published': 20090915,
  'tags': 'insurers mail',
  'text': 'Работаю в страховой компании. Любимые клиенты пишут:«На крышу мерседеса упала с балкона кошка. Cуществуют риски "падение предмета" и "атака животного". Что писать в страховом акте?»Претензионщики мрачно шутят, что если бы она упала и померла в полете — то было бы проще. Записали бы предметом.',
  'title': 'Атака одушевленного обьекта',
  'url': 'http://zadolba.li/story/5'},
 '_type': 'stories',
 '_version': 1,
 'found': True}

# Analytics